In [93]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import re
import random
import time

In [105]:
class pbp_scraping:
    def __init__(self, driver, game_id, url):
        self.url = url
        self.pbp_dict = {"game_id": game_id, "time": [], "quarter": [], "player_id": [],
                         "play_type": [], "secondary_player_id": [], "away score": [], "home score": [],
                         "rebound": [], "shot_type": [], "distance": [], "shot_score": []} #Gonna convert to dataframe later

        self.driver = driver
        self.quarter = "1"


    

    def get_player_id(self, play_col):
        # 'https://www.basketball-reference.com/players/c/curryst01.html'
        # Format of a link ^
        links = play_col.find_elements(By.XPATH, ".//a[contains(@href, '/players/')]")
        player_1 = "Team"
        player_2 = np.nan
        
        if len(links) >= 1:
            player_1 = links[0].get_attribute("href")
            player_1 = player_1.split("/")[-1]
            player_1 = player_1.split(".")[0]
        
        if len(links) == 2:
            player_2 = links[1].get_attribute("href")
            player_2 = player_2.split("/")[-1]
            player_2 = player_2.split(".")[0]


        return player_1, player_2
    
    def play_type(self, away_text, home_text):
        col_num = 0
        if away_text != " ":
            text = away_text
            col_num = 1
        if home_text != " ":
            text = home_text
            col_num = 5
        
        try:
            if (text == " "):
                return "No Play", col_num
            words = text.split()
            
            # Rebound check
            if (words[1] == "rebound"):
                return "Rebound", col_num
            # Shot made check
            if (words[2] == "makes"):
                # ADD FREE THROW CHECK
                return "Shot Made", col_num
            # Shot missed check
            if (words[2] == "misses"):
                return "Shot Missed", col_num
            # Turnover
            if (words[0] == "Turnover"):
                return "Turnover", col_num
            # Foul
            if ((words[1] == "foul") | (words[2] == "foul")):
                return "Foul", col_num
            # Enters
            if (words[2] == "enters"):
                return "Enters", col_num
            # Timeout
            if (words[3] == "timeout"):
                return "Timeout", col_num
            
            return "No Play", col_num
        
        except IndexError:
            return "No Play", col_num
        
    
    def get_shot_data(self, shot_text):
        shot_score = np.nan
        shot_type = np.nan
        distance = np.nan
        shot_info = shot_text.text.split()
        shot_score = shot_info[3]
        shot_type = shot_info[4]
        # distance_check = shot_text.text.split("(")[0]
        # distance = distance_check.split()[-1]
        distance_check = re.search(r"(\d+) ft", shot_text.text)
        distance = distance_check.group(0) if distance_check else np.nan

        return shot_score, shot_type, distance

    def get_rebound_type(self, rebound_text):
        rebound = np.nan

        rebound = rebound_text.text.split()[0] #Offensive or Defensive
        return rebound
        
    def append_to_dict(self, timer, player_id, play_type, secondary_player_id, 
                       away_score, home_score, rebound,
                       shot_type, distance, shot_score):
        self.pbp_dict["time"].append(timer)
        self.pbp_dict["quarter"].append(self.quarter)
        self.pbp_dict["player_id"].append(player_id)
        self.pbp_dict["play_type"].append(play_type)
        self.pbp_dict["secondary_player_id"].append(secondary_player_id)
        self.pbp_dict["rebound"].append(rebound)
        self.pbp_dict["shot_type"].append(shot_type)
        self.pbp_dict["distance"].append(distance)
        self.pbp_dict["shot_score"].append(shot_score)
        self.pbp_dict["away score"].append(away_score)
        self.pbp_dict["home score"].append(home_score)
        #Tada!
        
    def scrape_table(self):
        self.driver.get(self.url)
        time.sleep(random.uniform(0.5,3))
        play_table = self.driver.find_element(By.ID, "pbp")
        rows = play_table.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            #Check that it is a play
            if len(cols) == 6:
                play_type, col_num = self.play_type(cols[1].text, cols[5].text)
                if (play_type == "No Play"): #Move to the next row if we don't have a play
                    continue #This really only happens in extenuating circumstances and REALLY weird plays
                player_id, secondary_player_id = self.get_player_id(cols[col_num])
                timer = cols[0].text
                scores = cols[3].text.split("-")
                away_score = scores[0]
                home_score = scores[1]
                if (play_type in ["Shot Made", "Shot Missed"]):
                    shot_score, shot_type, distance = self.get_shot_data(cols[col_num])
                else:
                    shot_score, shot_type, distance = np.nan, np.nan, np.nan
                if (play_type=="Rebound"):
                    rebound = self.get_rebound_type(cols[col_num])
                else:
                    rebound = np.nan
                self.append_to_dict(timer, player_id, play_type, secondary_player_id, 
                                    away_score, home_score, rebound,
                                    shot_type, distance, shot_score)

            #check that quarter has ended
            if len(cols) == 2:
                quarter_text = cols[1].text
                if quarter_text.startswith("Start of"):
                    self.quarter = f"{quarter_text.split()[-2]} {quarter_text.split()[-1]}"
            

            


In [ ]:
driver = webdriver.Chrome()
# url = f"https://www.basketball-reference.com/boxscores/pbp/{Year}{Month}{Day}0{HomeTeam}.html"
url = "https://www.basketball-reference.com/boxscores/pbp/201602270OKC.html"
pbp_class = pbp_scraping(driver, 1, url, "Away", "Home")
pbp_class.scrape_table()


{'game_id': 1, 'time': ['11:50.0', '11:46.0', '11:28.0', '11:27.0', '11:19.0', '11:13.0', '11:11.0', '10:51.0', '10:26.0', '10:11.0', '10:10.0', '9:52.0', '9:30.0', '9:21.0', '9:20.0', '9:15.0', '9:15.0', '9:15.0', '9:15.0', '8:57.0', '8:57.0', '8:41.0', '8:41.0', '8:28.0', '8:16.0', '8:00.0', '8:00.0', '7:53.0', '7:51.0', '7:35.0', '7:34.0', '7:19.0', '7:16.0', '7:16.0', '7:16.0', '7:04.0', '7:04.0', '6:47.0', '6:32.0', '6:16.0', '6:06.0', '5:49.0', '5:48.0', '5:42.0', '5:42.0', '5:42.0', '5:42.0', '5:42.0', '5:42.0', '5:28.0', '5:28.0', '5:18.0', '5:18.0', '5:13.0', '5:13.0', '5:13.0', '5:13.0', '5:13.0', '4:54.0', '4:54.0', '4:45.0', '4:45.0', '4:38.0', '4:38.0', '4:38.0', '4:38.0', '4:38.0', '4:25.0', '4:06.0', '3:53.0', '3:45.0', '3:39.0', '3:36.0', '3:20.0', '3:20.0', '3:03.0', '3:02.0', '2:58.0', '2:42.0', '2:35.0', '2:34.0', '2:34.0', '2:34.0', '2:34.0', '2:21.0', '2:16.0', '1:59.0', '1:37.0', '1:37.0', '1:19.0', '1:10.0', '0:50.0', '0:31.0', '0:21.0', '0:21.0', '0:03.0', '0:01

In [106]:
head_options = webdriver.ChromeOptions()
head_options.add_argument("--disable-blink-features=AutomationControlled")
head_options.add_argument("--disable-gpu")
head_options.add_argument("--no-sandbox")
head_options.add_argument("--disable-extensions")
head_options.add_argument("--disable-dev-shm-usage")
head_options.add_argument("--disable-popup-blocking")
head_options.add_argument("--disable-notifications")
head_options.add_argument("--disable-infobars")
head_options.add_experimental_option("excludeSwitches", ["enable-automation"])
head_options.add_experimental_option('useAutomationExtension', False)
prefs = {"profile.managed_default_content_settings.images": 2}
head_options.add_experimental_option("prefs", prefs)


driver = webdriver.Chrome(options=head_options)

games = 0

#GET THE GAME IDS WE HAVEN'T DONE YET
done_ids = pd.read_csv("../data/PBPData.csv")["game_id"].unique().tolist()
all_game_ids = pd.read_csv("../data/BoxScoreData.csv")["Game_ID"].unique().tolist()
game_ids = [gid for gid in all_game_ids if gid not in done_ids]
game_ids = [str(gid) for gid in game_ids if gid > 10000]
game_ids.sort(reverse=True)

df = pd.read_csv("../data/ScheduleData2.csv")



for i in game_ids:
    fake_url = df.loc[df["game_id"] == i]["url"].values[0] #Get the url, need to change it up a bit though
    splitted = fake_url.split("/boxscores")

    url = f"{splitted[0]}/boxscores/pbp{splitted[1]}"
    try:
        pbp_class = pbp_scraping(driver, i, url)
        pbp_class.scrape_table()
        new_df = pd.DataFrame.from_dict(pbp_class.pbp_dict)
        new_df.to_csv("../data/PBPData.csv", mode='a', header=False, index=False)
        games += 1
        print(games)
    except Exception as e:
        print(f"Error on game {i}: {e}")
        continue





driver.quit()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [107]:
driver.quit()